In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)


In [5]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [10]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [25]:
import bs4
loader = WebBaseLoader(
    web_paths=("https://www.analyticsvidhya.com/blog/2021/07/recommendation-system-understanding-the-basic-concepts/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content-box", "wp-block-heading", "post-header")
        )
    )
)

In [26]:
# loading docs from some sources
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.analyticsvidhya.com/blog/2021/07/recommendation-system-understanding-the-basic-concepts/'}, page_content='\nRecommendation systems are AI-driven tools that analyze user preferences, behaviors, and data to provide personalized suggestions. Widely used in e-commerce, streaming services, social media, and more, they enhance user experience by recommending relevant products, movies, music, or content. These systems leverage algorithms to predict user interests, helping them discover new items while boosting engagement and satisfaction for businesses. From collaborative filtering to content-based and hybrid approaches, recommendation systems adapt to diverse needs.In this article their will be discuss on  various things on the recommendation system like what is the recommendation system? What are its use-cases? How many types of recommendation systems and metrics are used for it.\nThis article was published as a part of the\xa0Data Science Blogatho

In [27]:
# Dividing the loaded docs into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002832B14BC80>, search_kwargs={})

In [29]:
splits

[Document(metadata={'source': 'https://www.analyticsvidhya.com/blog/2021/07/recommendation-system-understanding-the-basic-concepts/'}, page_content='Recommendation systems are AI-driven tools that analyze user preferences, behaviors, and data to provide personalized suggestions. Widely used in e-commerce, streaming services, social media, and more, they enhance user experience by recommending relevant products, movies, music, or content. These systems leverage algorithms to predict user interests, helping them discover new items while boosting engagement and satisfaction for businesses. From collaborative filtering to content-based and hybrid approaches, recommendation systems adapt to diverse needs.In this article their will be discuss on  various things on the recommendation system like what is the recommendation system? What are its use-cases? How many types of recommendation systems and metrics are used for it.\nThis article was published as a part of the\xa0Data Science Blogathon'

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response = rag_chain.invoke({"input": "What are the types of recommendation systems?"})

In [34]:
response['answer']

'The main types are **content‑based filtering** and **collaborative filtering** (which includes user‑based and item‑based approaches). Some systems also combine these into hybrid models, but the core categories are content‑based and collaborative.'

In [35]:
## Adding conversational memory to the chatbot {Chat History}
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [38]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

In [39]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [40]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt
)

In [41]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002832B14BC80>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [42]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [43]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [44]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Recomendation system?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question2,"chat_history":chat_history})
print(response2['answer'])

Recommendation systems work by collecting and preparing user‑item data, selecting a model (e.g., content‑based, collaborative, or hybrid), training it, evaluating performance, and then deploying it while continuously monitoring and fine‑tuning. They boost engagement and sales across domains such as e‑commerce, streaming, social media, travel, and news by delivering personalized suggestions based on preferences, behavior, and similarity to other users. Their effectiveness can be measured with metrics like mean average precision at K, coverage, personalization, and intra‑list similarity.


In [45]:
chat_history

[HumanMessage(content='What is Recomendation system?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='A recommendation system is an information‑filtering algorithm that predicts how a user would rate or prefer an item and then suggests relevant items to them. It powers personalized suggestions such as movies on Netflix, products on Amazon, or books on Kindle. By analyzing user behavior and item attributes, it tailors content to improve user experience and drive engagement.', additional_kwargs={}, response_metadata={})]

In [46]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [47]:
conversational_rag_chain.invoke(
    {"input": "Why we use recommendation systems?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'We use recommendation systems to deliver personalized suggestions that boost sales, engagement, and user satisfaction across platforms such as e‑commerce, streaming, social media, news, advertising, and travel. By tailoring items to individual preferences, behavior, and similarities with other users, they help users discover relevant content more easily. This personalization enhances the overall user experience and drives business outcomes.'

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What if we don't use recommendation systems?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Without recommendation systems, users would receive generic, non‑personalized content, leading to lower engagement, satisfaction, and reduced sales or usage. Businesses would miss the ability to tailor offers, discover new user interests, and efficiently scale personalized experiences. Consequently, product discovery becomes harder and overall platform performance typically declines.'

In [50]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Why we use recommendation systems?', additional_kwargs={}, response_metadata={}), AIMessage(content='We use recommendation systems to deliver personalized suggestions that boost sales, engagement, and user satisfaction across platforms such as e‑commerce, streaming, social media, news, advertising, and travel. By tailoring items to individual preferences, behavior, and similarities with other users, they help users discover relevant content more easily. This personalization enhances the overall user experience and drives business outcomes.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What if we don't use recommendation systems?", additional_kwargs={}, response_metadata={}), AIMessage(content='Without recommendation systems, users would receive generic, non‑personalized content, leading to lower engagement, satisfaction, and reduced sales or usage. Businesses would miss the ability to tailor of